# Installation Etherpad-lite auf dem Pi

Grundlage dieser Anleitung ist https://github.com/ghoulmann/raspi-etherpad-lite

Zunächst die Paketedatenbank aktualisieren:

In [3]:
sudo apt-get -qq update

OK   http://mirrordirector.raspbian.org jessie InRelease
OK   http://mirrordirector.raspbian.org stretch InRelease
OK   http://archive.raspberrypi.org jessie InRelease
OK   http://mirrordirector.raspbian.org jessie/main armhf Packages
OK   http://mirrordirector.raspbian.org jessie/contrib armhf Packages
OK   http://mirrordirector.raspbian.org jessie/non-free armhf Packages
OK   http://archive.raspberrypi.org jessie/main armhf Packages
OK   http://mirrordirector.raspbian.org jessie/firmware armhf Packages
OK   https://deb.nodesource.com jessie InRelease
OK   https://apt.dockerproject.org raspbian-jessie InRelease
OK   https://deb.nodesource.com jessie/main Sources
OK   https://deb.nodesource.com jessie/main armhf Packages
OK   https://apt.dockerproject.org raspbian-jessie/main armhf Packages
Holen: 1 https://deb.nodesource.com jessie/main Translation-de_DE [162 B]
Holen: 2 https://apt.dockerproject.org raspbian-jessie/main Translation-de_DE [454 B]
Holen: 3 https://deb.nodesource.com je

Dann ggf. fehlende Pakete nachinstallieren.

In [5]:
sudo apt-get install -y gzip git-core curl python libssl-dev pkg-config build-essential

Paketlisten werden gelesen... Fertig
Abhängigkeitsbaum wird aufgebaut.       
Statusinformationen werden eingelesen.... Fertig
build-essential ist schon die neueste Version.
curl ist schon die neueste Version.
gzip ist schon die neueste Version.
libssl-dev ist schon die neueste Version.
pkg-config ist schon die neueste Version.
python ist schon die neueste Version.
Die folgenden NEUEN Pakete werden installiert:
  git-core
0 aktualisiert, 1 neu installiert, 0 zu entfernen und 35 nicht aktualisiert.
Es müssen 1.488 B an Archiven heruntergeladen werden.
Nach dieser Operation werden 21,5 kB Plattenplatz zusätzlich benutzt.
Holen: 1 http://mirrordirector.raspbian.org/raspbian/ jessie/main git-core all 1:2.1.4-2.1+deb8u3 [1.488 B]
Es wurden 1.488 B in 0 s geholt (25,2 kB/s).
debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
Vormals ni

Etherpad-lite benötigt node-js. Sollte installiert sein.

In [8]:
node --version

v6.10.3


Variablen mit dem Ort und dem Port. Danach wird etherpad-lite geklonet.

In [7]:
target=/opt/etherpad-lite
port=8080
#git-clone to /opt/etherpad-lite
sudo git clone git://github.com/ether/etherpad-lite.git $target

Klone nach '/opt/etherpad-lite'...
remote: Counting objects: 29259, done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 29259 (delta 4), reused 11 (delta 4), pack-reused 29247
Empfange Objekte: 100% (29259/29259), 18.85 MiB | 1.77 MiB/s, Fertig.
Löse Unterschiede auf: 100% (20877/20877), Fertig.
Prüfe Konnektivität... Fertig.


Vorlage kopieren und danach den Port anpassen.

In [9]:
#Create settings.json
sudo cp $target/settings.json.template $target/settings.json

In [10]:
#configure etherpad
sudo sed -i "s|9001|$port|" $target/settings.json

Die Abhängigkeiten mit Skript nachinstallieren.

In [11]:
#Make sure dependencies are up to date
sudo $target/bin/installDeps.sh

Ensure that all dependencies are up to date...  If this is the first time you have run Etherpad please be patient.
npm WARN deprecated node-uuid@1.4.8: Use uuid module insteadry.n2
npm WARN deprecated jade@0.26.3: Jade has been renamed to pug, please install the latest version of pug instead of jade
npm WARN deprecated graceful-fs@2.0.3: graceful-fs v3.0.0 and before will fail on node releases >= v7.0. Please update to graceful-fs@^4.0.0 as soon as possible. Use 'npm ls graceful-fs' to find it in the tree.
npm WARN deprecated minimatch@0.2.14: Please update to minimatch 3.0.2 or higher to avoid a RegExp DoS issue
npm WARN deprecated minimatch@1.0.0: Please update to minimatch 3.0.2 or higher to avoid a RegExp DoS issue
ep_etherpad-lite@1.6.1 /opt/etherpad-lite/srcrpad-lite: sill install runPostinstallTopLeslease u
├── async@0.9.0 
├── async-stacktrace@0.0.2 
├── channels@0.0.4 
├─┬ cheerio@0.20.0 
│ ├─┬ css-select@1.2.0 
│ │ ├── boolbase@1.0.0 
│ │ ├── css-what@2.1.0 
│ │ ├── domutils@

│ │ ├─┬ minimatch@3.0.3 
│ │ │ └─┬ brace-expansion@1.1.6 
│ │ │   ├── balanced-match@0.4.2 
│ │ │   └── concat-map@0.0.1 
│ │ └── path-is-absolute@1.0.1 
│ ├── graceful-fs@4.1.10 
│ ├── has-unicode@2.0.1 
│ ├── hosted-git-info@2.1.5 
│ ├── iferr@0.1.5 
│ ├── imurmurhash@0.1.4 
│ ├── inflight@1.0.6 
│ ├── inherits@2.0.3 
│ ├── ini@1.3.4 
│ ├─┬ init-package-json@1.9.4 
│ │ ├─┬ glob@6.0.4 
│ │ │ ├─┬ minimatch@3.0.3 
│ │ │ │ └─┬ brace-expansion@1.1.6 
│ │ │ │   ├── balanced-match@0.4.2 
│ │ │ │   └── concat-map@0.0.1 
│ │ │ └── path-is-absolute@1.0.0 
│ │ └── promzard@0.3.0 
│ ├─┬ JSONStream@1.2.1 
│ │ ├── jsonparse@1.2.0 
│ │ └── through@2.3.8 
│ ├── lockfile@1.0.2 
│ ├── lodash._baseindexof@3.1.0 
│ ├─┬ lodash._baseuniq@4.6.0 
│ │ ├── lodash._createset@4.0.3 
│ │ └── lodash._root@3.0.1 
│ ├── lodash._bindcallback@3.0.1 
│ ├── lodash._cacheindexof@3.0.2 
│ ├── lodash._createcache@3.1.2 
│ ├── lodash._getnative@3.9.1 
│ ├── lodash.clonedeep@4.5.0 
│ ├── lodash.restparam@3.6.1 
│ ├── lodash

│ │ ├── buffer-shims@1.0.0 
│ │ ├── core-util-is@1.0.2 
│ │ ├── isarray@1.0.0 
│ │ ├── process-nextick-args@1.0.7 
│ │ ├── string_decoder@0.10.31 
│ │ └── util-deprecate@1.0.2 
│ ├── readdir-scoped-modules@1.0.2 
│ ├── realize-package-specifier@3.0.3 
│ ├─┬ request@2.78.0 
│ │ ├── aws-sign2@0.6.0 
│ │ ├── aws4@1.5.0 
│ │ ├── caseless@0.11.0 
│ │ ├─┬ combined-stream@1.0.5 
│ │ │ └── delayed-stream@1.0.0 
│ │ ├── extend@3.0.0 
│ │ ├── forever-agent@0.6.1 
│ │ ├─┬ form-data@2.1.1 
│ │ │ └── asynckit@0.4.0 
│ │ ├─┬ har-validator@2.0.6 
│ │ │ ├─┬ chalk@1.1.3 
│ │ │ │ ├── ansi-styles@2.2.1 
│ │ │ │ ├── escape-string-regexp@1.0.5 
│ │ │ │ ├── has-ansi@2.0.0 
│ │ │ │ └── supports-color@2.0.0 
│ │ │ ├─┬ commander@2.9.0 
│ │ │ │ └── graceful-readlink@1.0.1 
│ │ │ ├─┬ is-my-json-valid@2.15.0 
│ │ │ │ ├── generate-function@2.0.0 
│ │ │ │ ├─┬ generate-object-property@1.2.0 
│ │ │ │ │ └── is-property@1.0.2 
│ │ │ │ ├── jsonpointer@4.0.0 
│ │ │ │ └── xtend@4.0.1 
│ │ │ └─┬ pinkie-promise@2.0.1 
│ │ │

│ │   └── ms@0.7.2 
│ ├─┬ socket.io-client@1.6.0 
│ │ ├── backo2@1.0.2 
│ │ ├── component-bind@1.0.0 
│ │ ├── component-emitter@1.2.1 
│ │ ├─┬ debug@2.3.3 
│ │ │ └── ms@0.7.2 
│ │ ├─┬ engine.io-client@1.8.0 
│ │ │ ├── component-emitter@1.2.1 
│ │ │ ├── component-inherit@0.0.3 
│ │ │ ├─┬ debug@2.3.3 
│ │ │ │ └── ms@0.7.2 
│ │ │ ├── has-cors@1.1.0 
│ │ │ ├── parsejson@0.0.3 
│ │ │ ├── parseqs@0.0.5 
│ │ │ ├── xmlhttprequest-ssl@1.5.3 
│ │ │ └── yeast@0.1.2 
│ │ ├── indexof@0.0.1 
│ │ ├── object-component@0.0.3 
│ │ ├─┬ parseuri@0.0.5 
│ │ │ └─┬ better-assert@1.0.2 
│ │ │   └── callsite@1.0.0 
│ │ └── to-array@0.1.4 
│ └─┬ socket.io-parser@2.3.1 
│   ├── component-emitter@1.1.2 
│   └── json3@3.3.2 
├─┬ supertest@1.2.0 
│ └─┬ superagent@1.8.5 
│   ├── component-emitter@1.2.1 
│   ├── cookiejar@2.0.6 
│   ├── extend@3.0.0 
│   ├─┬ form-data@1.0.0-rc3 
│   │ ├── async@1.5.2 
│   │ ├─┬ combined-stream@1.0.5 
│   │ │ └── delayed-stream@1.0.0 
│   │ └─┬ mime-types@2.1.15 
│   │   └── mime-db@1

  │ └── qs@1.2.2 
  ├── underscore.string@2.3.3 
  └── vargs@0.1.0 

Ensure jQuery is downloaded and up to date...
Clearing minified cache...
Ensure custom css/js files are created...


Ein Benuzter der etherpad-lite als Dienst laufen lässt.

ToDo: Das Anlegen eines Homeverzeichnises sollte für einen Systembenutzer nicht notwendig sein. Evtl auf /opt/etherpad-lite ändern.

In [12]:
#useradd system user named etherpad-lite
sudo useradd -MrU etherpad-lite
sudo mkdir /home/etherpad-lite
sudo chowm etherpad-lite:

In [13]:
#create log directory
sudo mkdir -p /var/log/etherpad-lite/

In [14]:
#permissions
sudo chown -R etherpad-lite:etherpad-lite $target/
sudo chown -R etherpad-lite:etherpad-lite /var/log/etherpad-lite/

Erstellen eines Start-Stop (initV) Skriptes.

In [16]:
#configure daemon
sudo su -c 'cat > /etc/init.d/etherpad-lite <<"EOF"
#!/bin/sh
### BEGIN INIT INFO
# Provides:          etherpad-lite
# Required-Start:    $local_fs $remote_fs $network $syslog
# Required-Stop:     $local_fs $remote_fs $network $syslog
# Default-Start:     2 3 4 5
# Default-Stop:      0 1 6
# Short-Description: starts etherpad lite
# Description:       starts etherpad lite using start-stop-daemon
### END INIT INFO
PATH="/usr/local/sbin:/usr/local/bin:/sbin:/bin:/usr/sbin:/usr/bin:/opt/node/bin"
LOGFILE="/var/log/etherpad-lite/etherpad-lite.log"
EPLITE_DIR="#target#"
EPLITE_BIN="bin/safeRun.sh"
USER="etherpad-lite"
GROUP="etherpad-lite"
DESC="Etherpad Lite"
NAME="etherpad-lite"
set -e
. /lib/lsb/init-functions
start() {
  echo "Starting $DESC... "
  
	start-stop-daemon --start --chuid "$USER:$GROUP" --background --make-pidfile --pidfile /var/run/$NAME.pid --exec $EPLITE_DIR/$EPLITE_BIN -- $LOGFILE || true
  echo "done"
}
#We need this function to ensure the whole process tree will be killed
killtree() {
    local _pid=$1
    local _sig=${2-TERM}
    for _child in $(ps -o pid --no-headers --ppid ${_pid}); do
        killtree ${_child} ${_sig}
    done
    kill -${_sig} ${_pid}
}
stop() {
  echo "Stopping $DESC... "
   while test -d /proc/$(cat /var/run/$NAME.pid); do
    killtree $(cat /var/run/$NAME.pid) 15
    sleep 0.5
  done
  rm /var/run/$NAME.pid
  echo "done"
}
status() {
  status_of_proc -p /var/run/$NAME.pid "" "etherpad-lite" && exit 0 || exit $?
}
case "$1" in
  start)
	  start
	  ;;
  stop)
    stop
	  ;;
  restart)
	  stop
	  start
	  ;;
  status)
	  status
	  ;;
  *)
	  echo "Usage: $NAME {start|stop|restart|status}" >&2
	  exit 1
	  ;;
esac
exit 0
EOF'

In [17]:
#specify the etherpad install directory
sudo sed -i "s|#target#|$target|" /etc/init.d/etherpad-lite

In [19]:
#Make daemon file executeable
sudo chmod +x /etc/init.d/etherpad-lite

In [20]:
#Configure as a service
sudo update-rc.d etherpad-lite defaults

In [23]:
sudo /etc/init.d/etherpad-lite start

Starting etherpad-lite (via systemctl): etherpad-lite.service.


In [25]:
sudo /etc/init.d/etherpad-lite status

● etherpad-lite.service - LSB: starts etherpad lite
   Loaded: loaded (/etc/init.d/etherpad-lite)
   Active: active (running) since Do 2017-06-08 09:53:37 CEST; 44s ago
  Process: 6006 ExecStart=/etc/init.d/etherpad-lite start (code=exited, status=0/SUCCESS)
   CGroup: /system.slice/etherpad-lite.service
           ├─6011 /bin/sh /opt/etherpad-lite/bin/safeRun.sh /var/log/etherpad...
           └─6122 sleep 10

Jun 08 09:53:37 pi3-xy etherpad-lite[6006]: Starting Etherpad Lite...
Jun 08 09:53:37 pi3-xy etherpad-lite[6006]: done
Jun 08 09:53:37 pi3-xy systemd[1]: Started LSB: starts etherpad lite.


In [28]:
nmap localhost


Starting Nmap 6.47 ( http://nmap.org ) at 2017-06-08 14:01 CEST
Nmap scan report for localhost (127.0.0.1)
Host is up (0.0016s latency).
Not shown: 994 closed ports
PORT     STATE SERVICE
22/tcp   open  ssh
80/tcp   open  http
3306/tcp open  mysql
8080/tcp open  http-proxy
8181/tcp open  unknown
8888/tcp open  sun-answerbook

Nmap done: 1 IP address (1 host up) scanned in 0.41 seconds


ToDo: Ändern auf mysql Datenbank.